In [15]:
import sknetwork as skn
from sknetwork.clustering import Louvain, get_modularity
from sknetwork.linalg import normalize
from sknetwork.utils import get_membership
from sknetwork.visualization import visualize_graph
import networkx as nx  
import torch
from torch_geometric.utils import to_scipy_sparse_matrix, to_networkx
from torch_geometric.data import Data
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
from IPython.display import SVG

In [16]:
graph_dir = os.path.join('..','..', 'src', 'data', 'graph_data.pt')
graph = torch.load(graph_dir, weights_only=False)
data = graph['data']

# Extracting adjacency matrix from graph torch file
adjacency_matrix = to_scipy_sparse_matrix(data.edge_index)

# Extracting nodes position
G_nx = to_networkx(data, to_undirected=False)
pos = nx.spring_layout(G_nx)
positions = np.array([pos[i] for i in range(len(pos))])

# Clusterizing the graph
louvain = Louvain()
labels = louvain.fit_predict(adjacency_matrix)
labels_unique, counts = np.unique(labels, return_counts=True)
print(labels_unique, counts)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23] [1111  922  760  480  320  217  160  140  103    6    4    3    2    2
    2    2    2    2    2    2    2    2    2    2]


In [17]:
get_modularity(adjacency_matrix, labels)

np.float64(0.5210337850402389)

In [18]:
adjacency_aggregate = louvain.aggregate_
avg = normalize(get_membership(labels).T)
position_aggregate = avg.dot(positions)
labels_unique, counts = np.unique(labels, return_counts=True)

In [19]:
unique_clusters = np.unique(labels)

cluster_nodes = {}
for c in unique_clusters:
    cluster = np.where(labels == c)[0]
    if len(cluster) > 5: # If there are at least 10 users in the same cluser, preserves it
        cluster_nodes[c] = cluster

In [20]:
# Clusterize community in cluster i
mapping = graph['idx2user']
exp_clusters = {}

for k, cluster in cluster_nodes.items():
    users = []
    for node_id in cluster:
        username = mapping.get(node_id)
        users.append(username)

    exp_clusters[k] = users

In [21]:
def build_subgraph(cluster_id, users, edge_df, save_path):
    print(f"Processing cluster : {cluster_id} with {len(users)} users.")
    users = set(users)
    edges = edge_df[edge_df['source'].isin(users) & edge_df['target'].isin(users)]

    if len(edges) == 0:
        return

    # Mapping id - users 
    user2idx = {uid: idx for idx, uid in enumerate(users)}
    idx2user = {idx: uid for uid, idx in user2idx.items()}

    # Creating edges and associated weights
    edges_index = torch.tensor([
        [user2idx[src] for src in edges['source']],
        [user2idx[dst] for dst in edges['target']],
    ], dtype=torch.long)

    edges_weight = torch.tensor(edges['weight'].values, dtype=torch.float)

    # Creating the subgraph aka the k-th cluster
    x = torch.eye(len(user2idx))
    data = Data(x=x, edge_index=edges_index, edge_weight=edges_weight)
    torch.save({
        'data': data,
        'idx2user': idx2user,
    }, save_path)

    return save_path

In [22]:
graph_dir = os.path.join('..','..', 'src', 'data', 'graph_data.pt')

def get_cluster_with_Louvain(subgraph_dir, threshold=0.55, min_users=5):
    subgraph = torch.load(subgraph_dir, weights_only=False)
    data = subgraph['data']

    # Extracting adjacency matrix of the subgraph
    adjacency_matrix = to_scipy_sparse_matrix(data.edge_index)

    louvain = Louvain()
    labels = louvain.fit_predict(adjacency_matrix)

    modularity = get_modularity(adjacency_matrix, labels)

    # This means that nodes are cohesive so there's no need to cluterize the subgraph more
    if modularity > threshold:
        return None
    
    print(f"Modularity: {modularity}")
    
    unique_clusters = np.unique(labels)
    cluster_nodes = {}

    for c in unique_clusters:
        cluster = np.where(labels == c)[0]
        if len(cluster) > min_users: # If there are at least 5 users in the same cluser, preserves it
            cluster_nodes[c] = cluster

    return cluster_nodes

In [23]:
def recursive_clustering(edge_df, subgraph_path, base_cluster_id, base_output_dir, depth=0, max_depth = 4):
    # Stopping cluster creation at max_depth
    if depth >= max_depth:
        return
    
    new_clusters = get_cluster_with_Louvain(subgraph_path)

    if new_clusters is None:
        return
    
    subgraph = torch.load(subgraph_path, weights_only=False)
    idx2user = subgraph['idx2user']

    for sub_id, node_ids in new_clusters.items():
        users = [idx2user[idx] for idx in node_ids]

        cluster_id = f"{base_cluster_id}_{sub_id}"
        save_path = os.path.join(base_output_dir, f"subgraph_{cluster_id}.pt")
        
        new_path = build_subgraph(cluster_id, users, edge_df, save_path=save_path)
        if new_path:
            recursive_clustering(
            edge_df=edge_df,
            subgraph_path=new_path,
            base_cluster_id=cluster_id,
            base_output_dir=base_output_dir,
            depth=depth + 1,
            max_depth=max_depth
        )

In [24]:
# Wrapper to start the recursion
def start_recursive_clustering(first_clusters, edges_path, base_output_dir, max_depth=4):
    edges = pd.read_csv(edges_path)

    for k, users in first_clusters.items():
        save_path = os.path.join(base_output_dir, f'subgraph_{k}.pt')
        new_cluster_dir = build_subgraph(k, users, edges, save_path)
        if new_cluster_dir is not None:
            recursive_clustering(
                subgraph_path=new_cluster_dir,
                base_cluster_id=k,
                edge_df=edges,
                base_output_dir=base_output_dir,
                depth=0,
                max_depth=max_depth
            )

In [25]:
edges_path = os.path.join('..', '..', 'src', 'data', 'edges.csv')
save_dir = os.path.join('..', '..', 'src', 'graph_dir', 'subgraph_dir')
os.makedirs(save_dir, exist_ok=True)

start_recursive_clustering(exp_clusters, edges_path, save_dir)



Processing cluster : 0 with 1111 users.
Modularity: 0.402435195618417
Processing cluster : 0_0 with 115 users.
Modularity: 0.5158648847604521
Processing cluster : 0_0_0 with 20 users.
Modularity: 0.507560815253123
Processing cluster : 0_0_0_0 with 6 users.
Modularity: 0.21999999999999997
Processing cluster : 0_0_1 with 18 users.
Modularity: 0.34829867674858217
Processing cluster : 0_0_2 with 17 users.
Modularity: 0.47346514047866817
Processing cluster : 0_0_2_0 with 6 users.
Modularity: 0.22000000000000008
Processing cluster : 0_0_2_1 with 6 users.
Modularity: 0.25
Processing cluster : 0_0_3 with 17 users.
Modularity: 0.38095238095238104
Processing cluster : 0_0_4 with 14 users.
Modularity: 0.5104
Processing cluster : 0_0_4_0 with 6 users.
Modularity: 0.2599999999999999
Processing cluster : 0_0_5 with 9 users.
Modularity: 0.35714285714285726
Processing cluster : 0_0_6 with 8 users.
Modularity: 0.2825
Processing cluster : 0_0_7 with 7 users.
Modularity: 0.29166666666666663
Processing cl